Zillow, what the heck? 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

The basics**strong text**

In [ ]:
import patsy
from tqdm import tqdm
import string
import time, os, pickle, gzip
import multiprocessing as mp
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss#, roc_auc_score
import sklearn


def prnt_updt(what_u_b_updtng, the_start):
    kardo_time = '{:*^32.22}'.format(time.strftime("%a %b %d %I:%M:%S %p"))
    elpsed = '{:,G}'.format(time.time() - the_start)
    print(kardo_time + '{:^32.32}'.format(what_u_b_updtng.upper() + ' SHIT!?') + '{:<32.32}'.format(elpsed))

load

In [ ]:
start_time = time.time()
df_train = pd.read_csv(r'../input/train_2016.csv')
df_prop = pd.read_csv(r'../input/properties_2016.csv')
prnt_updt("LOADED SHIT", start_time)


In [ ]:
df_sample = pd.read_csv(r'../input/sample_submission.csv')

In [ ]:
dft_len = len(df_train)
dfp_len = len(df_prop)
print([dft_len, dfp_len])

In [ ]:
print(df_train.head())
prnt_updt("PRINTED HEAD?", start_time)


In [ ]:
print(df_prop.head())
prnt_updt("PRINTED HEAD?", start_time)

In [ ]:
print(df_sample.head())
prnt_updt("PRINTED HEAD?", start_time)

In [ ]:
prnt_updt("is zillow messing with my", start_time)
print('null parse lids → ' + str((df_train['parcelid'].isnull()).sum()))
print('null parse lids → ' + str((df_prop['parcelid'].isnull()).sum()))

In [ ]:
df_merge_train_sample = df_train.merge(df_sample, left_on='parcelid', right_on='ParcelId', how='inner' )
prnt_updt("srsly zil? different case?", start_time)
print(df_merge_train_sample.head())

In [ ]:
print(len(df_train))
print(len(df_sample))
print(len(df_merge_train_sample))

In [ ]:
df_merge_prop_sample = df_prop.merge(df_sample, left_on='parcelid', right_on='ParcelId', how='inner' )
prnt_updt("srsly zil? different case?", start_time)
print(df_merge_train_sample.head())

In [ ]:
print(len(df_prop))
print(len(df_sample))
print(len(df_merge_prop_sample))

Interesting... 

In [ ]:
original_time = start_time
start_time = time.time()
df_merge_train_prop = df_train.merge(df_prop, left_on='parcelid', right_on='parcelid', how='inner' )
prnt_updt("srsly zillow?", start_time)
print(df_merge_train_sample.head())

In [ ]:
print(len(df_train))
print(len(df_prop))
print(len(df_merge_train_prop)) # just to make sure

In [ ]:
df_merge_train_prop.isnull()

In [ ]:
(df_merge_train_prop.isnull()).sum(axis=0)

In [ ]:
    prnt_updt('STARTING UP', start_time)
    # stupid zillow just make it one file you idiot
    df = df_merge_train_prop.copy()
    df['is_train'] = np.random.uniform(0, 1, len(df)) <= 0.80
    train_full, test = df[df['is_train']==True], df[df['is_train']==False]
    print('Number of observations in the training data:', len(train_full))
    print('Number of observations in the test data:',len(test))
    #clf = GradientBoostingClassifier(loss='exponential', learning_rate=0.01, n_estimators=500, max_depth=9, subsample=0.87, verbose=True)
    clf = RandomForestClassifier(n_estimators=500, oob_score=True, n_jobs=4, min_samples_split=4, verbose=True)
    prnt_updt('DONE INITIALIZING', start_time)

In [ ]:
 prnt_updt('getting sleepy now', original_time)

In [ ]:
1561.89/60

In [ ]:
##### get the features by copy paste
print(df_merge_train_prop.columns)

In [ ]:
FEATURES = pd.indexes.base.Index(['parcelid', 'logerror', 'transactiondate', 'airconditioningtypeid',
       'architecturalstyletypeid', 'basementsqft', 'bathroomcnt', 'bedroomcnt',
       'buildingclasstypeid', 'buildingqualitytypeid', 'calculatedbathnbr',
       'decktypeid', 'finishedfloor1squarefeet',
       'calculatedfinishedsquarefeet', 'finishedsquarefeet12',
       'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50',
       'finishedsquarefeet6', 'fips', 'fireplacecnt', 'fullbathcnt',
       'garagecarcnt', 'garagetotalsqft', 'hashottuborspa',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
       'propertycountylandusecode', 'propertylandusetypeid',
       'propertyzoningdesc', 'rawcensustractandblock', 'regionidcity',
       'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt',
       'storytypeid', 'threequarterbathnbr', 'typeconstructiontypeid',
       'unitcnt', 'yardbuildingsqft17', 'yardbuildingsqft26', 'yearbuilt',
       'numberofstories', 'fireplaceflag', 'structuretaxvaluedollarcnt',
       'taxvaluedollarcnt', 'assessmentyear', 'landtaxvaluedollarcnt',
       'taxamount', 'taxdelinquencyflag', 'taxdelinquencyyear',
       'censustractandblock'],
      dtype='object')
#FEATURES.drop('parcelid', 1)
#FEATURES.drop('logerror', 1)
print(FEATURES)

In [ ]:
FEATURES = pd.indexes.base.Index([ 'transactiondate', 'airconditioningtypeid',
       'architecturalstyletypeid', 'basementsqft', 'bathroomcnt', 'bedroomcnt',
       'buildingclasstypeid', 'buildingqualitytypeid', 'calculatedbathnbr',
       'decktypeid', 'finishedfloor1squarefeet',
       'calculatedfinishedsquarefeet', 'finishedsquarefeet12',
       'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50',
       'finishedsquarefeet6', 'fips', 'fireplacecnt', 'fullbathcnt',
       'garagecarcnt', 'garagetotalsqft', 'hashottuborspa',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
       'propertycountylandusecode', 'propertylandusetypeid',
       'propertyzoningdesc', 'rawcensustractandblock', 'regionidcity',
       'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt',
       'storytypeid', 'threequarterbathnbr', 'typeconstructiontypeid',
       'unitcnt', 'yardbuildingsqft17', 'yardbuildingsqft26', 'yearbuilt',
       'numberofstories', 'fireplaceflag', 'structuretaxvaluedollarcnt',
       'taxvaluedollarcnt', 'assessmentyear', 'landtaxvaluedollarcnt',
       'taxamount', 'taxdelinquencyflag', 'taxdelinquencyyear',
       'censustractandblock'],
      dtype='object')

In [ ]:
train_full[FEATURES][:1]

In [ ]:
#df_merge_train_prop[:20]
sk_le = sklearn.preprocessing.LabelEncoder()
np.isinstance(train_full[FEATURES], float)
print("hello")
#ground_truth_full = train_full['logerror'].values
#clf.fit(train_full[FEATURES], ground_truth_full)